In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [22]:
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Dona": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [23]:
data_train['Title'] = data_train['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
data_train['Title'] = data_train['Title'].map(Title_Dictionary)

In [24]:
fill_mean = lambda g: g.fillna(g.mean())
data_train['Age'] = data_train.groupby(['Sex', 'Title', 'Pclass'])['Age'].apply(fill_mean)

In [25]:
data_train.Age = data_train.groupby(['Sex', 'Title', 'Pclass'])['Age'].apply(fill_mean)

data_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
dtype: int64

In [26]:
from sklearn.model_selection import train_test_split

data_train['Sex'] = data_train['Sex'].map({'male':1, 'female':0})


X_all = data_train[['Pclass','Sex','Age']]
y_all = data_train['Survived']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

In [27]:
data_train = pd.get_dummies(data_train, columns=['Sex','Embarked'])

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()

parameters = {'n_estimators': [4, 6, 9], 
              'max_features': ['log2', 'sqrt', 'auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }


acc_scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

clf = grid_obj.best_estimator_

clf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=5, max_features='sqrt',
                       n_estimators=4)

In [29]:
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))

0.8044692737430168


In [32]:
from sklearn.model_selection import KFold

kf = KFold(891, n_splits=10, shuffle=False)    
outcomes = []
    
fold = 0
for train_index, test_index in enumerate(kf.split(data_train)):
    fold += 1
    X_train, X_test = X_all.values[train_index], X_all.values[test_index]
    y_train, y_test = y_all.values[train_index], y_all.values[test_index]
    
    clf.fit(X_train, y_train)
    
    predictions = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, predictions)
    outcomes.append(accuracy)
    
    print("Fold {0} accuracy: {1}".format(fold, accuracy))     
mean_outcome = np.mean(outcomes)
print("\n\nMean Accuracy: {0}".format(mean_outcome)) 

AttributeError: 'Series' object has no attribute 'value'